In [87]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
import tensorflow as tf
import numpy as np
if __name__ == "__main__":
    sys.path.append("../../")

from notebooks_src.configs import configs
from notebooks_src.postprocessing.utils import get_int_tensor_shape, sort_some_lists_of_tensors

from notebooks_src.box_processing.match import match_boxes
from notebooks_src.box_processing.encode import encode
from notebooks_src.box_processing.unpack import unpack_net_output, split_box_class

from notebooks_src.box_processing.decode import decode
#from notebooks_src.postprocessing.ssd import detected_bboxes
#from notebooks_src.postprocessing.bboxes import bboxes_matching_batch
from notebooks_src.postprocessing.utils import reshape_list
from notebooks_src.metrics.utils import precision_recall, average_precision_voc07, average_precision_voc12, EpochMetrics
from notebooks_src.configs import configs
from notebooks_src.box_processing.tf_box_util import zero_out_negatives, make_box_coords_relative, make_actual_gt_box_mask, ious_with_reference_boxes






In [117]:
def calc_map(y_true, y_preds):
    cgt_bboxes = process_ground_truth_for_map(y_true)
    cbboxes, cscores = postprocess_boxes_for_map_or_plotting(y_preds)
    #tp,fp,scores, calc_tp_fp(cbboxes, gt_bboxes, cscores)
    #return dict(tp=tp, fp=fp, scores=scores)
    

In [118]:
def process_ground_truth_for_map(y_true):
    gt_bboxes, labels = split_box_class(y_true)
    gt_bboxes = zero_out_negatives(gt_bboxes)
    gt_bboxes = make_box_coords_relative(gt_bboxes)
    
    cgt_bboxes = organize_gt_by_class(gt_bboxes, labels)
    return cgt_bboxes

In [119]:
def postprocess_boxes_for_map_or_plotting(y_preds):
    detections = unpack_net_output(y_preds)
    pred_bboxes, scores = decode(detections)
    cbboxes, cscores = organize_by_class(pred_bboxes, scores)
    cbboxes, cscores = filter_out_low_confidence_boxes(cbboxes, cscores)
    cbboxes, cscores = get_top_k_boxes_for_each_class(cbboxes, cscores, k=configs["select_top_k"])
    cbboxes, cscores = do_nms_on_boxes(cbboxes, cscores,
                                 nms_threshold=configs["nms_threshold"],
                                 keep_top_k=configs["keep_top_k"])
    return cbboxes, cscores

In [108]:
def do_nms_on_boxes(cbboxes, cscores, nms_threshold,keep_top_k):
    for c in cbboxes.keys():
        bboxes = cbboxes[c]
        scores = cscores[c]
        cbboxes[c], cscores[c] = do_nms_batch(bboxes, scores, nms_threshold,keep_top_k)
    return cbboxes, cscores    

def do_nms_batch(bboxes, scores, nms_threshold,keep_top_k):
    batch_size = bboxes.get_shape()[0].value
    #the ol'reverse zip 
    bboxes, scores = zip(*[nms_single_example(bboxes[ex], scores[ex], nms_threshold, keep_top_k) for ex in range(batch_size) ])
    bboxes = tf.stack(bboxes)
    scores = tf.stack(scores)
    return bboxes, scores

def nms_single_example(bboxes, scores, nms_threshold=0.5, keep_top_k=200):
    idxes = tf.image.non_max_suppression(bboxes, scores,
                                         keep_top_k, nms_threshold)
    scores = tf.gather(scores, idxes)
    
    bboxes = tf.gather(bboxes, idxes)
    return bboxes, scores

In [109]:
def get_top_k_boxes_for_each_class(cbboxes, cscores, k=400):
    for c in cscores.keys():
        scores = cscores[c]
        bboxes = cbboxes[c]
        cscores[c],idxes = tf.nn.top_k(scores, k=k, sorted=True)
        cbboxes[c] = gather_2d(bboxes, idxes)
    return cbboxes, cscores

In [110]:
def gather_2d(tensor, idxes):
    """takes in tensor of size nxk and idxes of size nxp where p <= k and does a gather
        works great when we want subselect indices that vary over the batch"""
    return tf.stack([tf.gather(tensor[i], idxes[i]) for i in range(tensor.get_shape()[0].value)])
    

In [111]:
def organize_gt_by_class(gt_bboxes, gt_labels):
    """these are numpy arrays for now"""
    cgt_bboxes = {}
    for c in range(configs["num_classes"]):
        bboxes_list = []
        for i in range(configs["batch_size"]):
            bboxes = gt_bboxes[i]
            labels = gt_labels[i]
            class_mask = tf.expand_dims(tf.cast(tf.equal(labels, c), dtype=bboxes.dtype), axis=-1)
            bboxes_list.append(bboxes * class_mask)
        cgt_bboxes[c] = tf.stack(bboxes_list)
    return cgt_bboxes
            
        

In [112]:
def organize_by_class(bboxes, scores):
    num_classes = configs["num_classes"]
    cbb = {}
    csc = {}
    for c in range(num_classes):
        cbb[c] = bboxes
        csc[c] = scores[:,:,c]
    return cbb, csc
        
        
def filter_out_low_confidence_boxes(cbboxes, cscores, low_conf_threshold=0.01):
    """ bboxes and scores are organized by class now (they are dicts key:class ind, value:boxes or scores)"""
    for c in cbboxes.keys():
        scores = cscores[c]
        bboxes = cbboxes[c]
        low_conf_mask = tf.cast(tf.greater_equal(scores, 0.01), dtype=scores.dtype)
        cscores[c] = scores * low_conf_mask
        cbboxes[c] = bboxes * tf.expand_dims(tf.cast(low_conf_mask, dtype=bboxes.dtype), axis=-1)
    return cbboxes, cscores

In [113]:
def calc_batch_metrics(y_true, y_preds):
    bboxes, labels = split_box_class(y_true)
    encoded_boxes_dict, encoded_labels_dict = encode(bboxes,labels)
    loc_dict, log_dict, pred_dict = unpack_net_output(y_preds)
    


    
    # Performing post-processing on CPU: loop-intensive, usually more efficient.
    #with tf.device('/device:CPU:0'):
    localizations = decode(localizations)
    
    # get top k  predicted boxes after nms
    rscores, rbboxes = detected_bboxes(predictions, localizations,
                            select_threshold=configs["select_threshold"],
                            nms_threshold=configs["nms_threshold"],
                            clipping_bbox=None,
                            top_k=configs["select_top_k"],
                            keep_top_k=configs["keep_top_k"])
    
    #match the predicted boxes to ground truth boxes and compute the TP and FP statistics.
    num_gbboxes, tp, fp, rscores = \
    bboxes_matching_batch(rscores.keys(), rscores, rbboxes,
                              glabels, gbboxes, gdifficults,
                              matching_threshold=configs["matching_threshold"])
    
    return num_gbboxes, tp, fp, rscores    

In [114]:
def calc_ap_one_class():
    num_classes = configs["num_classes"]
    
    
    num_gbboxes = tf.placeholder(dtype=tf.int32)
    tp = tf.placeholder(dtype=tf.bool)
    fp = tf.placeholder(dtype=tf.bool)
    rscores = tf.placeholder(dtype=tf.float32)
    num_detections = tf.placeholder(dtype=tf.int32)
    
    
    # Add to summaries precision/recall values.
    aps_voc07 = {}
    aps_voc12 = {}

    # Precison and recall values.
    prec, rec = precision_recall(num_gbboxes, num_detections,
                                tp, fp, rscores )

    # Average precision VOC07.
    v = average_precision_voc07(prec, rec)
    aps_voc07 = v

    # Average precision VOC12.
    v = average_precision_voc12(prec, rec)

    aps_voc12 = v
        
    return aps_voc12,[num_gbboxes, num_detections, tp, fp, rscores]


In [115]:
if __name__ == "__main__":
    with tf.Session() as sess:

        import h5py
        y_true = tf.placeholder(tf.float32,shape=(5,15,5))
        box = h5py.File(configs["tr_data_file"])["boxes"][323:328]
        shapes = [
                    (5, 6, 9, 54),
                    (5, 3, 5, 36),
                    (5, 96, 144, 36),
                    (5, 24, 36, 54),
                    (5, 12, 18, 54),
                    (5, 48, 72, 54),
                    (5, 1, 1, 36)]

        y_true = box #.astype("float32")
        y_preds = [tf.random_normal(mean=0.0,stddev=3.,shape=shape) for shape in shapes]
        bi = calc_map(y_true, y_preds)
        best_ind = sess.run(bi)

InvalidArgumentError: WhereOp : Unhandled input dimensions: 0
	 [[Node: Where_86 = Where[_device="/job:localhost/replica:0/task:0/cpu:0"](LogicalAnd_260)]]

Caused by op u'Where_86', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/python2.7/dist-packages/tornado/ioloop.py", line 887, in start
    handler_func